In [ ]:
import os
from PIL import Image, ImageEnhance, ImageOps
import random
from random import randint
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
import tensorflow as tf; 
print(tf.version)

In [ ]:
input_aspect = 42.0 / 30.0 
input_width = 28
input_height = 28
output_classes = 11

In [ ]:
%matplotlib inline

def randomize_brightness(image):
    enhancer = ImageEnhance.Brightness(image)    
    return enhancer.enhance(random.uniform(0.6, 1.1))

def read_image(filePath):
    image = Image.open(filePath)
    
    image = randomize_brightness(image)
    
    max_size = max(image.width, image.height)
    col = image.getpixel((0, 0))
    image = ImageOps.pad(image, (max_size, int(max_size * input_aspect)), color = col)

    # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.resize
    image = image.resize((input_width, input_height), Image.LANCZOS)

    # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.convert
    image = image.convert("L")

    image = np.asarray(image)

    return np.asarray(image).astype(np.float32) / 255.

def read_images(path):
    files = os.listdir(path)    
    files = [file for file in files if file[-4:] == ".png"]
    random.shuffle(files)
    images = []
    for file in files:
        try:
            images.append(read_image(os.path.join(path, file)))
        except OSError:
            pass
    
    return images

max_images_per_digit = 0
input_dict = {}
for i in range(output_classes):
    images = read_images("images/labeled/" + str(i))
    print("Digit: " + str(i) + "; Images: " + str(len(images)))
    input_dict[i] = images
    max_images_per_digit = max(max_images_per_digit, len(images))

print("Equalizing size of each class...")    

xy_arr = []
for i in range(output_classes):
    images = input_dict[i]
    for image in images:
        xy_arr.append([image, i])
    repeats = max_images_per_digit - len(images)
    print("Digit: " + str(i) + "; +" + str(repeats))
    for n in range(repeats):
        image = images[randint(0, len(images) - 1)]
        xy_arr.append([image, i])
        
print("Total number of digits: " + str(len(xy_arr)))

random.shuffle(xy_arr)

test_samples = 20
x_train_arr = []
y_train_arr = []
x_test_arr = []
y_test_arr = []
idx = 0
for xy in xy_arr:
    if idx < test_samples:
        x_test_arr.append(xy[0])
        y_test_arr.append(xy[1])  
    else:
        x_train_arr.append(xy[0])
        y_train_arr.append(xy[1])
    idx = idx + 1
    
# convert to numpy types
x_train = np.asarray(x_train_arr)#.astype(np.float32) / 255.
y_train = to_categorical(y_train_arr)
x_test = []
y_test = []
if test_samples > 0:
    x_test = np.asarray(x_test_arr)#.astype(np.float32) / 255.
    y_test = to_categorical(y_test_arr)
#print("x_train: " + str(x_train))
#print("y_train: " + str(y_train))
#print("##########################")
#print("x_test: " + str(x_test))
#print("y_test: " + str(y_test))

In [ ]:
def createModel():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation = 'relu', input_shape = (input_height, input_width, 1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_classes, activation = 'softmax'))

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])
    return model

In [ ]:
model = createModel()
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
use_generator = True
batch_size = 1024

if use_generator:
    gen = ImageDataGenerator(
        width_shift_range=3, 
        height_shift_range=3, 
        zoom_range=0.1, 
        horizontal_flip=False)
        #rotation_range=10,
        #brightness_range=[0.5, 1.0])
    model.fit_generator(gen.flow(
        x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1), 
        y_train, batch_size=batch_size, shuffle=True), epochs=300, workers=24, steps_per_epoch=len(x_train) / batch_size)
else:
    model.fit(
        x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1),
        y_train,
        epochs=2000,
        batch_size=batch_size)

In [ ]:
def predict(img):
    result = model.predict(img.reshape(1, img.shape[0], img.shape[1], 1))
    pred = np.argmax(result, axis = 1)[0]
    convidence = result[0][pred]
    return (pred, convidence)

def predict_test(idx):
    result = predict(x_test[idx])
    pred = result[0]
    convidence = result[1]
    expect = y_test_arr[idx]
    correct = expect == pred    
    print("correct: " + str(correct) + 
          "; expected: " + str(expect) + 
          "; predicted: " + str(pred) + 
          "; confidence: " + str(round(convidence, 2)))
    if not correct or convidence < 0.9:
        plt.imshow(x_test[idx])
        plt.show()
for i in range(len(x_test_arr)):
    predict_test(i)

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
filename = "ocr_model_" + str(input_width) + "x" + str(input_height) + "_c" + str(output_classes)
print("writing " + filename + ".tflite")
open(filename + ".tflite", "wb").write(tflite_model)
print("Now call the following in git bash: xxd -i " + filename + ".tflite > ../src/" + filename + ".c")